In [72]:
import itertools
import math
from itertools import islice
from pprint import pprint

import numpy as np
import opentrons.simulate

protocol = opentrons.simulate.get_protocol_api("2.12")
protocol.home()

%run opentrons_titrate_functions.ipynb

/Users/timkench/.opentrons/robot_settings.json not found. Loading defaults
/Users/timkench/.opentrons/deck_calibration.json not found. Loading defaults


In [73]:
tiprack_20a = protocol.load_labware("opentrons_96_tiprack_20ul", location="7")
tiprack_20b = protocol.load_labware("opentrons_96_tiprack_20ul", location="8")
tiprack_20c = protocol.load_labware("opentrons_96_tiprack_20ul", location="9")

tiprack_300a = protocol.load_labware("opentrons_96_tiprack_300ul", location="10")
tiprack_300b = protocol.load_labware("opentrons_96_tiprack_300ul", location="11")

m20 = protocol.load_instrument(
    "p20_multi_gen2", "left", tip_racks=[tiprack_20a, tiprack_20b, tiprack_20c]
)
m300 = protocol.load_instrument(
    "p300_multi_gen2", "right", tip_racks=[tiprack_300a, tiprack_300b]
)

labware_96w_200ul = "biorad_96_wellplate_200ul_pcr"
labware_12w_22ml = "nest_96_wellplate_2ml_deep"
labware_384w_20ul = "appliedbiosystemsmicroamp_384_wellplate_40ul"

res_plate_ot = protocol.load_labware(labware_12w_22ml, "1")
reaction_plate_ot = protocol.load_labware(labware_96w_200ul, "2")
nmr_rack_ot = protocol.load_labware(labware_96w_200ul, "3")

In [74]:
# set up plates

res_plate = ResPlate(reagents={"CHCl3": 0, "standard": 1})
res_plate.assign_ot(res_plate_ot)

reaction_plate = GeneralPlate(active_cols=(0, 12))
reaction_plate.assign_ot(reaction_plate_ot)

NMR_rack = GeneralPlate(map_to=reaction_plate)
NMR_rack.assign_ot(nmr_rack_ot)
NMR_rack.make_transfer_map()

# set up pipettes and protocol

m20_c = CustomPipette(m20, name="m20_c")
m300_c = CustomPipette(m300, name="m300_c")

CP = CustomProtocol(
    solvent_location=res_plate.reagents["CHCl3"], m20=m20_c, m300=m300_c
)

# do transfer

m20_c.set_solvent("buffer")
CP.transfer(plate=NMR_rack, vol=20, top_dispense="Yes")
m300_c.set_solvent("buffer")
CP.add(
    plate=NMR_rack, source=res_plate.reagents["CHCl3"], vol=250, top_dispense="Yes"
)
CP.add(
    plate=NMR_rack, source=res_plate.reagents["standard"], vol=250, top_dispense="Yes"
)

This will use the transfer map of the specified plate to carry out a transfer using the m300_c pipette.


Do you want to continue? (Y/N):  y


continuing...
done
This will add the specified source to the active columns of the plate using the m300_c pipette.


Do you want to continue? (Y/N):  y


continuing...
done
This will add the specified source to the active columns of the plate using the m300_c pipette.


Do you want to continue? (Y/N):  y


continuing...
done


In [71]:
for line in protocol.commands():
    print(line)

Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 7
Aspirating 10.8 uL from A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 6.08 uL/sec
Delaying for 0 minutes and 1.0 seconds
Moving to A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2
Dispensing 10.0 uL into A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3 at 6.08 uL/sec
Delaying for 0 minutes and 1.0 seconds
Dispensing 0.88 uL into A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 6.08 uL/sec
Delaying for 0 minutes and 1.0 seconds
Moving to A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2
Dropping tip into A1 of Opentrons Fixed Trash on 12
Picking up tip from A2 of Opentrons 96 Tip Rack 20 µL on 7
Aspirating 10.8 uL from A2 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 6.08 uL/sec
Delaying for 0 minutes and 1.0 seconds
Moving to A2 of Bio-Rad 96 Well Plate 200 µL PCR on 2
Dispensing 10.0 uL into A2 of Bio-Rad 96 Well Plate 200 µL PCR on 3 at 6.08 uL/sec
Delaying for 0 minutes and 1.0 seconds
Dispensing 0.88 uL into A2 of Bio-Rad 96 Well Plate 200 µL